In [17]:
#import

import os
import glob
import numpy
import numpy as np
import pymrmr
import random
import pandas
import roc_utils as ru
import joblib
import matplotlib.pyplot as plt

#sklearn
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.tree import ExtraTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import roc_curve, auc, precision_score, recall_score, f1_score, accuracy_score, roc_auc_score,classification_report
from sklearn.utils import resample
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay 
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score


from scipy import stats
from pathlib import Path
from ReliefF import ReliefF
from itertools import cycle
from joblib import Parallel, delayed
from matplotlib import pyplot

numpy.set_printoptions(precision=2)
NCPU=30

In [18]:
def MultiCollinearityEliminator(dataframe, threshold):
    variables = set(dataframe.columns)
    high_correlated_variables = set()
    for i in variables:
        for j in (variables - high_correlated_variables):
            if i == j:
                continue
            else:
                rank, p_value = stats.spearmanr(dataframe[i], dataframe[j])
                if abs(rank) >= threshold:
                    high_correlated_variables.add(i)
                    break
    df = dataframe.drop(dataframe[list(high_correlated_variables)], axis=1)
    return df




#Defining function for confusion matrix plot
n_classes=2
def plot_confusion_matrix(y_true, y_pred, classes=n_classes,dataset='Test',
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    if not title:
        if normalize:
            title = ''
        else:
            title = ''  
    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')  
    #Print Confusion matrix
    fig, ax = plt.subplots(figsize=(7,7))
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')
    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")
    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    if normalize:
        fig.savefig(f'{Directory}//{Folder}//{Feature_Selection}//{Path(file).stem}_{model}_{dataset}_NCM.jpg',
               dpi=300, facecolor='w', edgecolor='w',
               format='jpg', transparent=False, bbox_inches=None, pad_inches=0.1)
    else:
        fig.savefig(f'{Directory}//{Folder}//{Feature_Selection}//{Path(file).stem}_{model}_{dataset}_CM.jpg',
               dpi=300, facecolor='w', edgecolor='w',
               format='jpg', transparent=False, bbox_inches=None, pad_inches=0.1)
    plt.close()
    return ax  



def roc_plot(y_true,y_score,dataset='Train'):
        n_samples = 1000
        ru.plot_roc_bootstrap(X=y_score, y=y_true, pos_label=1,
                      n_bootstrap=n_samples,show_ti=True,show_ci=True,
                      title='');
        fpr, tpr, _ = roc_curve(y_true, y_score,pos_label=1)
        roc_auc= auc(fpr, tpr)
        # First aggregate all false positive rates

        # # Plot all ROC curves
        # lw=2
        # plt.figure()
        # plt.plot(
        #     fpr,
        #     tpr,
        #     label="AUC = {0:0.2f}".format(roc_auc),
        #     color="deeppink",
        #     linestyle=":",
        #     linewidth=4,
        # )

        # plt.plot([0, 1], [0, 1], "k--", lw=lw)
        # plt.xlim([0.0, 1.0])
        # plt.ylim([0.0, 1.05])
        # plt.xlabel("False Positive Rate")
        # plt.ylabel("True Positive Rate")
        pyplot.savefig(f'{Directory}//{Folder}//{Feature_Selection}//{Path(file).stem}_{model}_{dataset}_ROC.jpg', dpi=600, facecolor='w', edgecolor='w',
                    orientation='portrait', format='jpg', transparent=False, bbox_inches=None, pad_inches=0.1)
        #plt.show()
        pyplot.close()
        return roc_auc

In [19]:
os.chdir("/home/max/Documents/Classification/Data/WP")
Files_List = glob.glob('*.xlsx')
Train_Results_Matrix_all = pandas.DataFrame(columns=['set','model', "auc","accuracy","precision","recall","f1score"],
                                        index=range(0))
Test_Results_Matrix_all = Train_Results_Matrix_all.copy()
Validation_Results_Matrix_all = Train_Results_Matrix_all.copy()

for file in Files_List:
    print(file)
    DataSet = pandas.read_excel(file)
    n_classes=['GS<=7','GS>7']

    #n_classes=['GS=6','GS>=7']
    #
    # Delete Redundent Columns

    # Split DataSet to Train, Test, and Validation based on center
    Train_Test_Centers = [1, 2]
    Validation_Centers = [3]
    Train_Test_DataSet = DataSet.loc[DataSet["CENTER"].isin(Train_Test_Centers)]
    Validation_DataSet = DataSet.loc[DataSet["CENTER"].isin(Validation_Centers)]
 
    Train_DataSet, Test_DataSet = train_test_split(Train_Test_DataSet, train_size=0.80, test_size=0.2, stratify=Train_Test_DataSet["Label"])
    # Save Train. Test, and Validation Set
    Folder = Path(file).stem
    Directory = os.getcwd()
    if (os.path.exists(os.path.join(Directory, Folder)) is False):
        os.mkdir(os.path.join(Directory, Folder))
    Train_DataSet.to_csv(f'{Directory}//{Folder}//{Folder}_Train.csv', index=False, header=True)
    Test_DataSet.to_csv(f'{Directory}//{Folder}//{Folder}_Test.csv', index=False, header=True)
    Validation_DataSet.to_csv(f'{Directory}//{Folder}//{Folder}_Validation.csv', index=False, header=True)

    Train_Target_Var = Train_DataSet.loc[:, Train_DataSet.columns == "Label"]
    Train_Target_Var.index = range(len(Train_Target_Var))
    Train_Predictor_Vars = Train_DataSet.loc[:, Train_DataSet.columns.drop(['CENTER', 'Gleason', "Label"])]
    Train_Predictor_Vars.index = range(len(Train_Predictor_Vars))

    Test_Target_Var = Test_DataSet.loc[:, Test_DataSet.columns == "Label"]
    Test_Target_Var.index = range(len(Test_Target_Var))
    Test_Predictor_Vars = Test_DataSet.loc[:, Test_DataSet.columns.drop(['CENTER', 'Gleason', "Label"])]
    Test_Predictor_Vars.index = range(len(Test_Predictor_Vars))

    Validation_Target_Var = Validation_DataSet.loc[:, Validation_DataSet.columns == "Label"]
    Validation_Target_Var.index = range(len(Validation_Target_Var))
    Validation_Predictor_Vars = Validation_DataSet.loc[:,
                                Validation_DataSet.columns.drop(['CENTER', 'Gleason', "Label"])]
    Validation_Predictor_Vars.index = range(len(Validation_Predictor_Vars))

GA-PSMA_Multicentric(WP).xlsx


# Feature Selection

In [20]:
#1 MRMR Feature Selection

Normalized_Train_Data = pandas.DataFrame(
        preprocessing.StandardScaler().fit_transform(Train_Predictor_Vars),
        columns=Train_Predictor_Vars.columns)
Normalized_Train_Data = pandas.concat([Train_Target_Var, Normalized_Train_Data], axis=1)
MRMR_Selected_Features = pymrmr.mRMR(Normalized_Train_Data, 'MIQ', 7)
Train_Data_MRMR_Selected_Features = Train_Predictor_Vars[MRMR_Selected_Features]
Test_Data_MRMR_Selected_Features = Test_Predictor_Vars[MRMR_Selected_Features]
Validation_Data_MRMR_Selected_Features = Validation_Predictor_Vars[MRMR_Selected_Features]
Scaler = preprocessing.StandardScaler()
Scaler.fit(Train_Data_MRMR_Selected_Features)
Normalized_Train_Data_MRMR_Selected_Features = pandas.DataFrame(
        Scaler.transform(Train_Data_MRMR_Selected_Features),
        columns=Train_Data_MRMR_Selected_Features.columns)
Normalized_Test_Data_MRMR_Selected_Features = pandas.DataFrame(
        Scaler.transform(Test_Data_MRMR_Selected_Features),
        columns=Test_Data_MRMR_Selected_Features.columns)
Normalized_Validation_Data_MRMR_Selected_Features = pandas.DataFrame(
        Scaler.transform(Validation_Data_MRMR_Selected_Features),
        columns=Validation_Data_MRMR_Selected_Features.columns)
Folder = Path(file).stem + '//MRMR'
Directory = os.getcwd()
if (os.path.exists(os.path.join(Directory, Folder)) is False):
        os.mkdir(os.path.join(Directory, Folder))
Train_Data_MRMR_Selected_Features.to_csv(
        f'{Directory}//{Folder}//{Path(file).stem}_Train_MRMR.csv',
        index=False, header=True)
Test_Data_MRMR_Selected_Features.to_csv(
        f'{Directory}//{Folder}//{Path(file).stem}_Test_MRMR.csv',
        index=False, header=True)
Validation_Data_MRMR_Selected_Features.to_csv(
        f'{Directory}//{Folder}//{Path(file).stem}_Validation_MRMR.csv',
        index=False, header=True)




 *** This program and the respective minimum Redundancy Maximum Relevance (mRMR) 
     algorithm were developed by Hanchuan Peng <hanchuan.peng@gmail.com>for
     the paper 
     "Feature selection based on mutual information: criteria of 
      max-dependency, max-relevance, and min-redundancy,"
      Hanchuan Peng, Fuhui Long, and Chris Ding, 
      IEEE Transactions on Pattern Analysis and Machine Intelligence,
      Vol. 27, No. 8, pp.1226-1238, 2005.


*** MaxRel features ***
Order 	 Fea 	 Name 	 Score
1 	 23 	 DIS_ExcessKurtosis 	 0.142
2 	 22 	 DIS_Kurtosis 	 0.142
3 	 7 	 SUV_Q3 	 0.137
4 	 56 	 NGLDM_Contrast 	 0.125
5 	 2 	 SUV_mean 	 0.122
6 	 50 	 GLRLM_LRLGE 	 0.110
7 	 64 	 GLZLM_LZLGE 	 0.107

*** mRMR features *** 
Order 	 Fea 	 Name 	 Score
1 	 23 	 DIS_ExcessKurtosis 	 0.142
2 	 46 	 GLRLM_LGRE 	 4.459
3 	 66 	 GLZLM_GLNU 	 2.746
4 	 12 	 SUV_peakSphere0.5mL.1 	 2.645
5 	 25 	 DIS_peakSphere1mL 	 1.377
6 	 7 	 SUV_Q3 	 1.386
7 	 29 	 DIS_HISTO_Energy 	 1.008


In [21]:
#2 ANOVA Features Selection
df = MultiCollinearityEliminator(Train_Predictor_Vars, 0.9)
skb=SelectKBest(f_classif, k=7).fit(df, numpy.ravel(Train_Target_Var)).get_support(1)
ANOVA_Selected_Features = list(df.columns[skb])
Train_Data_ANOVA_Selected_Features = Train_DataSet[ANOVA_Selected_Features]
Test_Data_ANOVA_Selected_Features = Test_DataSet[ANOVA_Selected_Features]
Validation_Data_ANOVA_Selected_Features = Validation_DataSet[ANOVA_Selected_Features]
Scaler = preprocessing.StandardScaler()
Scaler.fit(Train_Data_ANOVA_Selected_Features)
Normalized_Train_Data_ANOVA_Selected_Features = pandas.DataFrame(
        Scaler.transform(Train_Data_ANOVA_Selected_Features),
        columns=Train_Data_ANOVA_Selected_Features.columns)
Normalized_Test_Data_ANOVA_Selected_Features = pandas.DataFrame(
        Scaler.transform(Test_Data_ANOVA_Selected_Features),
        columns=Test_Data_ANOVA_Selected_Features.columns)
Normalized_Validation_Data_ANOVA_Selected_Features = pandas.DataFrame(
        Scaler.transform(Validation_Data_ANOVA_Selected_Features),
        columns=Validation_Data_ANOVA_Selected_Features.columns)
Folder = Path(file).stem + '//ANOVA'
Directory = os.getcwd()
if (os.path.exists(os.path.join(Directory, Folder)) is False):
        os.mkdir(os.path.join(Directory, Folder))
Train_Data_ANOVA_Selected_Features.to_csv(
        f'{Directory}//{Folder}//{Path(file).stem}_Train_ANOVA.csv',
        index=False, header=True)
Test_Data_ANOVA_Selected_Features.to_csv(
        f'{Directory}//{Folder}//{Path(file).stem}_Test_ANOVA.csv',
        index=False, header=True)
Validation_Data_ANOVA_Selected_Features.to_csv(
        f'{Directory}//{Folder}//{Path(file).stem}_Validation_ANOVA.csv',
        index=False, header=True)

In [22]:
#3 KruskalWallis Features Selection
df = MultiCollinearityEliminator(Train_Predictor_Vars, 0.9)
New_Train_DataSet = pandas.concat([Train_Target_Var, df], axis=1)

label, idx = np.unique(New_Train_DataSet['Label'], return_inverse=True)

# make a list of arrays containing the y-values corresponding to each unique label
New_Train_DataSet.drop("Label", inplace=True, axis=1)
# use `*` to unpack the list as a sequence of arguments to `stats.kruskal`
KruskalWallis_Selected_Features = []
for predictor in range(New_Train_DataSet.shape[1]):
        groups = [New_Train_DataSet.iloc[:, predictor][idx == i] for i, l in enumerate(label)]
        if (stats.kruskal(*groups).pvalue < 0.05):
            print(stats.kruskal(*groups).pvalue)
            KruskalWallis_Selected_Features.append(New_Train_DataSet.iloc[:, predictor].name)
if len(KruskalWallis_Selected_Features)<1:
        KruskalWallis_Selected_Features = []
        for predictor in range(New_Train_DataSet.shape[1]):
            groups = [New_Train_DataSet.iloc[:, predictor][idx == i] for i, l in enumerate(label)]
            if (stats.kruskal(*groups).pvalue.pvalue < 0.2):
                KruskalWallis_Selected_Features.append(New_Train_DataSet.iloc[:, predictor].name)
       
Train_Data_KruskalWallis_Selected_Features = Train_DataSet[KruskalWallis_Selected_Features]
Test_Data_KruskalWallis_Selected_Features = Test_DataSet[KruskalWallis_Selected_Features]
Validation_Data_KruskalWallis_Selected_Features = Validation_DataSet[KruskalWallis_Selected_Features]
Scaler = preprocessing.StandardScaler()
Scaler.fit(Train_Data_KruskalWallis_Selected_Features)
Normalized_Train_Data_KruskalWallis_Selected_Features = pandas.DataFrame(
        Scaler.transform(Train_Data_KruskalWallis_Selected_Features),
        columns=Train_Data_KruskalWallis_Selected_Features.columns)
Normalized_Test_Data_KruskalWallis_Selected_Features = pandas.DataFrame(
        Scaler.transform(Test_Data_KruskalWallis_Selected_Features),
        columns=Test_Data_KruskalWallis_Selected_Features.columns)
Normalized_Validation_Data_KruskalWallis_Selected_Features = pandas.DataFrame(
        Scaler.transform(Validation_Data_KruskalWallis_Selected_Features),
        columns=Validation_Data_KruskalWallis_Selected_Features.columns)
Folder = Path(file).stem + '//KruskalWallis'
Directory = os.getcwd()
if (os.path.exists(os.path.join(Directory, Folder)) is False):
        os.mkdir(os.path.join(Directory, Folder))
Train_Data_KruskalWallis_Selected_Features.to_csv(
        f'{Directory}//{Folder}//{Path(file).stem}_Train_KruskalWallis.csv',
        index=False, header=True)
Test_Data_KruskalWallis_Selected_Features.to_csv(
        f'{Directory}//{Folder}//{Path(file).stem}_Test_KruskalWallis.csv',
        index=False, header=True)
Validation_Data_KruskalWallis_Selected_Features.to_csv(
        f'{Directory}//{Folder}//{Path(file).stem}_Validation_KruskalWallis.csv',
        index=False, header=True)

0.0005640439771781853
0.00016361406665971886
0.0017645622392370329
0.009036071341573668
0.03155703142714589


In [23]:
#4 Relief Feature selection
df = MultiCollinearityEliminator(Train_Predictor_Vars, 0.90)
Relief_Feature_Selection = ReliefF(n_neighbors=20)
Relief_Feature_Selection.fit(df.to_numpy(), numpy.ravel(Train_Target_Var))
Ten_Top_Features_list = list(Relief_Feature_Selection.top_features)[0:10]
Train_Data_Relief_Selected_Features = Train_Predictor_Vars.iloc[:, Ten_Top_Features_list]
Test_Data_Relief_Selected_Features = Test_Predictor_Vars.iloc[:, Ten_Top_Features_list]
Validation_Data_Relief_Selected_Features = Validation_Predictor_Vars.iloc[:, Ten_Top_Features_list]
Scaler = preprocessing.StandardScaler()
Scaler.fit(Train_Data_Relief_Selected_Features)
Normalized_Train_Data_Relief_Selected_Features = pandas.DataFrame(
        Scaler.transform(Train_Data_Relief_Selected_Features),
        columns=Train_Data_Relief_Selected_Features.columns)
Normalized_Test_Data_Relief_Selected_Features = pandas.DataFrame(
        Scaler.transform(Test_Data_Relief_Selected_Features),
        columns=Test_Data_Relief_Selected_Features.columns)
Normalized_Validation_Data_Relief_Selected_Features = pandas.DataFrame(
        Scaler.transform(Validation_Data_Relief_Selected_Features),
        columns=Validation_Data_Relief_Selected_Features.columns)
Folder = Path(file).stem + '//Relief'
Directory = os.getcwd()
if (os.path.exists(os.path.join(Directory, Folder)) is False):
        os.mkdir(os.path.join(Directory, Folder))
Train_Data_Relief_Selected_Features.to_csv(
        f'{Directory}//{Folder}//{Path(file).stem}_Train_Relief.csv',
        index=False, header=True)
Test_Data_Relief_Selected_Features.to_csv(
        f'{Directory}//{Folder}//{Path(file).stem}_Test_Relief.csv',
        index=False, header=True)
Validation_Data_Relief_Selected_Features.to_csv(
        f'{Directory}//{Folder}//{Path(file).stem}_Validation_Relief.csv',
        index=False, header=True)

# Models: Logistic Regression, Linear Discriminant Analysis, Extra Trees,Random Forest, K-Nearest Neighbor

In [24]:
# K-fold CrossValidation
CV = StratifiedKFold(n_splits=5, shuffle=True,random_state=42)


#1 Logistic Regression classifire
Logit_Algorithm = LogisticRegression(random_state=2)
Logit_Params_Space = dict()
Logit_Params_Space['solver'] = ['newton-cg', 'lbfgs', 'liblinear']
Logit_Algorithm = GridSearchCV(Logit_Algorithm, Logit_Params_Space, cv=CV, scoring='accuracy', n_jobs=NCPU)



#2 Linear Discriminant Analysis
LDA_Algorithm = LinearDiscriminantAnalysis()

#3 ExtraTree
ExtraTree_Algorithm = ExtraTreeClassifier(random_state=0)
ExtraTree_params_space = dict()
ExtraTree_params_space['criterion'] = ['gini', 'entropy']
ExtraTree_params_space['max_depth'] = [6, 8, 10, 12, 14, 16, 18, 20, 22, 24]
ExtraTree_params_space['min_samples_split'] = [4, 6, 8, 10]
ExtraTree_params_space['max_features'] = ['sqrt', 5, 6, 8,10]
ExtraTree_Algorithm = RandomizedSearchCV(ExtraTree_Algorithm, ExtraTree_params_space, cv=CV, n_iter=150,
                                             scoring='accuracy', n_jobs=NCPU)



#4 RandomForest
RF_Algorithm = RandomForestClassifier(random_state=3)
RF_params_space = dict()
RF_params_space['n_estimators'] = numpy.arange(50, 500, 50)
RF_params_space['criterion'] = ['gini', 'entropy']
RF_params_space['max_depth'] = [6, 8, 10, 12, 14, 16, 18, 20, 22, 24]
RF_params_space['min_samples_split'] = [4, 6, 8, 10]
RF_params_space['max_samples'] = [0.2, 0.3, 0.4, 0.5, 0.6]
RF_params_space['max_features'] = ['sqrt', 5, 6, 8]
RF_Algorithm = RandomizedSearchCV(RF_Algorithm, RF_params_space, cv=CV, n_iter=150, scoring='accuracy', n_jobs=NCPU)
    

#5 k-nearest neighbors
KNN_Algorithm = KNeighborsClassifier()
KNN_params_space = dict()
KNN_params_space['n_neighbors'] = [3, 5, 7, 9]
KNN_params_space['weights'] = ['uniform', 'distance']
KNN_params_space['algorithm'] = ['auto', 'ball_tree', 'kd_tree', 'brute']
KNN_Algorithm = RandomizedSearchCV(KNN_Algorithm, KNN_params_space, cv=CV, n_iter=30, scoring='accuracy', n_jobs=NCPU)

# Models_List & Metrics

In [25]:
Models_List = \
        [
            'RF_MRMR', 'RF_ANOVA', 'RF_KruskalWallis', 'RF_Relief',

            'LDA_MRMR', 'LDA_ANOVA', 'LDA_KruskalWallis',  'LDA_Relief',

            'KNN_MRMR', 'KNN_ANOVA', 'KNN_KruskalWallis',  'KNN_Relief',
           
            'Logit_MRMR', 'Logit_ANOVA', 'Logit_KruskalWallis','Logit_Relief',

            'ExtraTree_MRMR', 'ExtraTree_ANOVA', 'ExtraTree_KruskalWallis', 'ExtraTree_Relief',    
        ]
Metrics_list = [ "auc","accuracy","precision","recall","f1score"]


In [26]:
#creates DataFrames to store performance metrics (AUC, accuracy, precision, recall, F1 score) 

Train_Results_Matrix = pandas.DataFrame(columns=['model', "auc","accuracy","precision","recall","f1score"],index=range(20))
Train_Results_Matrix.iloc[:, 0] = Models_List

Test_Results_Matrix = Train_Results_Matrix.copy()
Test_Results_Matrix.iloc[:, 0] = Models_List

Validation_Results_Matrix = Train_Results_Matrix.copy()
Validation_Results_Matrix.iloc[:, 0] = Models_List

In [27]:
# 1. Algoritm_Dict - associates various machine learning algorithms 
Algoritm_Dict = dict()
Algoritm_Dict["RF"] = RF_Algorithm
Algoritm_Dict["LDA"] = LDA_Algorithm
Algoritm_Dict["KNN"] = KNN_Algorithm
Algoritm_Dict["Logit"] = Logit_Algorithm
Algoritm_Dict["ExtraTree"] = ExtraTree_Algorithm


# 2. Features_Dict - organizes selected features from different feature selection methods 
Features_Dict = dict()
Features_Dict["MRMR"] = \
        {"Train": Normalized_Train_Data_MRMR_Selected_Features,
         "Test": Normalized_Test_Data_MRMR_Selected_Features,
         "Validation": Normalized_Validation_Data_MRMR_Selected_Features}
Features_Dict["ANOVA"] = \
        {"Train": Normalized_Train_Data_ANOVA_Selected_Features,
         "Test": Normalized_Test_Data_ANOVA_Selected_Features,
         "Validation": Normalized_Validation_Data_ANOVA_Selected_Features}
Features_Dict["KruskalWallis"] = \
        {"Train": Normalized_Train_Data_KruskalWallis_Selected_Features,
         "Test": Normalized_Test_Data_KruskalWallis_Selected_Features,
         "Validation": Normalized_Validation_Data_KruskalWallis_Selected_Features}
Features_Dict["Relief"] = \
        {"Train": Normalized_Train_Data_Relief_Selected_Features,
         "Test": Normalized_Test_Data_Relief_Selected_Features,
         "Validation": Normalized_Validation_Data_Relief_Selected_Features}
   
Folder = Path(file).stem 
for model, i in zip(Models_List, range(len(Models_List))):
        print(model)

RF_MRMR
RF_ANOVA
RF_KruskalWallis
RF_Relief
LDA_MRMR
LDA_ANOVA
LDA_KruskalWallis
LDA_Relief
KNN_MRMR
KNN_ANOVA
KNN_KruskalWallis
KNN_Relief
Logit_MRMR
Logit_ANOVA
Logit_KruskalWallis
Logit_Relief
ExtraTree_MRMR
ExtraTree_ANOVA
ExtraTree_KruskalWallis
ExtraTree_Relief


In [28]:
# 1. Splits the 'model' variable into two parts: the algorithm name and feature selection method.
Algoritm, Feature_Selection = model.split('_')
# 2. Retrieves the corresponding algorithm from Algoritm_Dict and selected features from Features_Dict.
BaseAlgorithm = Algoritm_Dict[f'{Algoritm}']
Features = Features_Dict[f'{Feature_Selection}']
# 3. Fits the selected machine learning algorithm to the training features and target variable.
Final_Model = BaseAlgorithm.fit(Features["Train"], Train_Target_Var.values.ravel())
# 4. Saves the trained model as a pickle file in a specified directory, incorporating the feature selection method and model name into the filename.
joblib.dump(Final_Model,  f'{Directory}//{Folder}//{Feature_Selection}//'+Path(file).stem+'_'+model+'.pkl', )

/home/max/.local/lib/python3.10/site-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


['/home/max/Documents/Classification/Data/WP//GA-PSMA_Multicentric(WP)//Relief//GA-PSMA_Multicentric(WP)_ExtraTree_Relief.pkl']

In [29]:
# Predict the probabilities for the training dataset
y_pred_train = Final_Model.predict_proba(Features["Train"])

# Plot the confusion matrix for the test dataset
plot_confusion_matrix(Test_Target_Var.values,Final_Model.predict(Features["Test"]),classes=n_classes, dataset='Test')



# Calculate the AUC for the training dataset
auc_train_micro=roc_plot(y_true=Train_Target_Var,y_score=y_pred_train[:,1],dataset='Train')
Train_Results_Matrix.iloc[i, 1] = auc_train_micro


# Generate the classification report for the test dataset    
cr_test=classification_report( Test_Target_Var.values,Final_Model.predict(Features["Test"]) ,target_names=n_classes)

# Save the classification report to a text file
with open(f'{Directory}//{Folder}//{Feature_Selection}//{Path(file).stem}_{model}_Test.txt', 'w') as f:
            f.write(cr_test)

# Predict the probabilities for the test dataset
y_pred_test = Final_Model.predict_proba(Features["Test"])

# Calculate the AUC for the test dataset
auc_test_micro=roc_plot(y_true=Test_Target_Var,y_score=y_pred_test[:,1],dataset='Test')
Test_Results_Matrix.iloc[i, 1] = auc_test_micro

# Save the test predictions to a CSV file
y_pred_test_s=pandas.DataFrame(y_pred_test)
y_pred_test_s.to_csv(f'{Directory}//{Folder}//{Feature_Selection}//{Path(file).stem}_{model}_Test.csv')

# Predict the probabilities for the validation dataset
y_pred_valid = Final_Model.predict_proba(Features["Validation"])

# Plot the confusion matrix for the validation dataset
plot_confusion_matrix(Validation_Target_Var.values,Final_Model.predict(Features["Validation"]) ,classes= n_classes, dataset='Validation')

# Generate the classification report for the validation dataset
cr_valid=classification_report( Validation_Target_Var.values,Final_Model.predict(Features["Validation"]) ,target_names=n_classes)

# Save the classification report to a text file
with open(f'{Directory}//{Folder}//{Feature_Selection}//{Path(file).stem}_{model}_Validation.txt', 'w') as f:
            f.write(cr_valid)
auc_valid_micro=roc_plot(y_true=Validation_Target_Var,y_score=y_pred_valid[:,1],dataset='Validation')

# Save the validation predictions to a CSV file
y_pred_valid_s=pandas.DataFrame(y_pred_valid)
y_pred_valid_s.to_csv(f'{Directory}//{Folder}//{Feature_Selection}//{Path(file).stem}_{model}_Validation.csv')


# Store evaluation metrics for training, test, and validation
Validation_Results_Matrix.iloc[i, 1] =auc_valid_micro
Train_Results_Matrix.iloc[i, 2] = accuracy_score(Train_Target_Var, Final_Model.predict(Features["Train"]))
Test_Results_Matrix.iloc[i, 2] = accuracy_score(Test_Target_Var, Final_Model.predict(Features["Test"]))
Train_Results_Matrix.iloc[i, 3] = precision_score(Train_Target_Var, Final_Model.predict(Features["Train"]))
Test_Results_Matrix.iloc[i, 3] = precision_score(Test_Target_Var, Final_Model.predict(Features["Test"]))
Train_Results_Matrix.iloc[i, 4] = recall_score(Train_Target_Var, Final_Model.predict(Features["Train"]))
Test_Results_Matrix.iloc[i, 4] = recall_score(Test_Target_Var, Final_Model.predict(Features["Test"]))
Train_Results_Matrix.iloc[i, 5] = f1_score(Train_Target_Var, Final_Model.predict(Features["Train"]))
Test_Results_Matrix.iloc[i, 5] = f1_score(Test_Target_Var, Final_Model.predict(Features["Test"]))


# Store evaluation metrics for the validation dataset     
Validation_Results_Matrix.iloc[i, 2] = accuracy_score(Validation_Target_Var, Final_Model.predict(Features["Validation"]))
Validation_Results_Matrix.iloc[i, 3] = precision_score(Validation_Target_Var, Final_Model.predict(Features["Validation"]))
Validation_Results_Matrix.iloc[i, 4] = recall_score(Validation_Target_Var, Final_Model.predict(Features["Validation"]))
Validation_Results_Matrix.iloc[i, 5] = f1_score(Validation_Target_Var, Final_Model.predict(Features["Validation"]))


# Save the results matrices to CSV files
Train_Results_Matrix.to_csv(f'{os.getcwd()}//{Path(file).stem}//Train_Results.csv', index=False, header=True)
Test_Results_Matrix.to_csv(f'{os.getcwd()}//{Path(file).stem}//Test_Results.csv', index=False, header=True)
Validation_Results_Matrix.to_csv(f'{os.getcwd()}//{Path(file).stem}//Validation_Results.csv', index=False, header=True)

# Add a column indicating the dataset name
Train_Results_Matrix.insert(loc=0, column='set', value=Path(file).stem)
Test_Results_Matrix.insert(loc=0, column='set', value=Path(file).stem)
Validation_Results_Matrix.insert(loc=0, column='set', value=Path(file).stem)


# Concatenate the results matrices for all datasets
Train_Results_Matrix_all=pandas.concat([Train_Results_Matrix_all,Train_Results_Matrix], axis=0)
Test_Results_Matrix_all=pandas.concat([Test_Results_Matrix_all,Test_Results_Matrix], axis=0)
Validation_Results_Matrix_all=pandas.concat([Validation_Results_Matrix_all,Validation_Results_Matrix], axis=0)

# Save the concatenated results matrices to CSV files
Train_Results_Matrix_all.to_csv('Train_Results.csv', index=False, header=True)
Test_Results_Matrix_all.to_csv('Test_Results.csv', index=False, header=True)
Validation_Results_Matrix_all.to_csv('Validation_Results.csv', index=False, header=True)

Confusion matrix, without normalization
Confusion matrix, without normalization
